### 1. Neo4j Aura DB 환경설정

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph(
    url = os.getenv("NEO4J_URI"),
    username = os.getenv("NEO4J_USERNAME")
)

In [6]:
def reset_database(graph):
    """데이터베이트 초기화 하기"""
    # 모든 노드와 관계 삭제
    graph.query("MATCH (n) DETACH DELETE n")

    # 모든 제약조건 삭제
    constraints = graph.query("SHOW CONSTRAINTS")
    for constraint in constraints:
        constraint_name = constraint.get("name")
        if constraint_name:
            graph.query(f"DROP CONSTRAINT {constraint_name}")
    
    # 모든 인덱스 삭제
    indexes = graph.query("SHOW INDEXES")
    for index in indexes : 
        index_name = index.get("name")
        index_type = index.get("type")
        if index_name and index_type != "CONSTRAINT":
            graph.query(f"DROP INDEX {index_name}")
    
    print("데이터베이스가 초기화 되었습니다.")

#데이터베이스 초기화
reset_database(graph)

데이터베이스가 초기화 되었습니다.


In [7]:
# 데이터 모델 정의
cypher_query = """
// Person 노드 생성
CREATE (p1:Person {name : '홍길동', age : 35, city : '부산'})
CREATE (p2:Person {name : '김철수', age : 28, city : '수원'})
CREATE (p3:Person {name : '이영희', age : 32, city : '구미'})
CREATE (p4:Person {name : '권두현', age : 30, city : '구미'})
CREATE (p5:Person {name : '서윤주', age : 30, city : '서울'})
CREATE (p6:Person {name : '박규리', age : 27, city : '서울'}) 
CREATE (p7:Person {name : '변지민', age : 25, city : '서울'})
CREATE (p8:Person {name : '김하경', age : 29, city : '수원'})

// 관심분야 노드 생성
CREATE (i1:Interest {name: '영화'})
CREATE (i2:Interest {name: '강아지'})
CREATE (i3:Interest {name: '고양이'})
CREATE (i4:Interest {name: '뮤지컬'})
CREATE (i5:Interest {name: '여행'})

// 장소 노드 생성
CREATE (l1: Location {name: '남산타워', city:'서울'})
CREATE (l2: Location {name: '해운대', city:'부산'})
CREATE (l3: Location {name: '광화문', city:'서울'})
CREATE (l4: Location {name: '행궁동', city:'수원'})

// KNOWS 관계 생성 (사람들 간의 친구 관계)
CREATE (p1)-[:KNOWS {since:2010}]->(p2)
CREATE (p1)-[:KNOWS {since:2015}]->(p3)
CREATE (p1)-[:KNOWS {since:2012}]->(p7)
CREATE (p2)-[:KNOWS {since:2018}]->(p6)
CREATE (p3)-[:KNOWS {since:2017}]->(p4)
CREATE (p3)-[:KNOWS {since:2020}]->(p5)
CREATE (p4)-[:KNOWS {since:2024}]->(p5)
CREATE (p6)-[:KNOWS {since:2024}]->(p8)
CREATE (p7)-[:KNOWS {since:2014}]->(p3)
CREATE (p7)-[:KNOWS {since:2024}]->(p8)

// LIKES 관계 생성 (사람과 관심분야 간의 관계)
CREATE (p1)-[:LIKES {rating:2}]->(i1)
CREATE (p1)-[:LIKES {rating:1}]->(i5)
CREATE (p2)-[:LIKES {rating:5}]->(i3)
CREATE (p3)-[:LIKES {rating:4}]->(i2)
CREATE (p3)-[:LIKES {rating:5}]->(i4)
CREATE (p4)-[:LIKES {rating:3}]->(i1)
CREATE (p4)-[:LIKES {rating:5}]->(p5)
CREATE (p5)-[:LIKES {rating:3}]->(i2)
CREATE (p6)-[:LIKES {rating:4}]->(i3)
CREATE (p7)-[:LIKES {rating:5}]->(i4)
CREATE (p8)-[:LIKES {rating:3}]->(i5)

// VISITED 관계 생성 (사람과 장소 간의 관계)
CREATE (p1)-[:VISITED {date: '2023-03-15', rating :4}]->(l1)
CREATE (p1)-[:VISITED {date: '2024-05-20', rating :5}]->(l2)
CREATE (p2)-[:VISITED {date: '2021-04-10', rating :3}]->(l2)
CREATE (p3)-[:VISITED {date: '2025-06-05', rating :5}]->(l3)
CREATE (p4)-[:VISITED {date: '2024-07-12', rating :4}]->(l4)
CREATE (p4)-[:VISITED {date: '2023-02-28', rating :5}]->(l1)
CREATE (p5)-[:VISITED {date: '2022-08-03', rating :4}]->(l3)
CREATE (p6)-[:VISITED {date: '2025-01-17', rating :5}]->(l2)
CREATE (p7)-[:VISITED {date: '2024-09-22', rating :3}]->(l1)
CREATE (p8)-[:VISITED {date: '2023-10-08', rating :4}]->(l2)
"""

graph.query(cypher_query)

[]

### 2. 가변길이 경로 (Variable-length paths)
* 가변 길이 경로는 관계의 길이를 가변적으로 지정할 수 있는 기능 입니다.


In [ ]:
cypher_query="""
//홍길동으로부터 1~3단계로 연결된 모든 사람을 찾는 쿼리
MATCH(p1:Person {name:'홍길동'})-[r:KNOWS*1..3]->(p2:Person)
RETURN p1.name AS 시작점, [rel IN r | type(rel)+ '('+rel.since+')'] AS 관계, p2.name AS 도착점

"""

graph.query(cypher_query)

[{'시작점': '홍길동', '관계': ['KNOWS(2012)'], '도착점': '변지민'},
 {'시작점': '홍길동', '관계': ['KNOWS(2012)', 'KNOWS(2024)'], '도착점': '김하경'},
 {'시작점': '홍길동', '관계': ['KNOWS(2012)', 'KNOWS(2014)'], '도착점': '이영희'},
 {'시작점': '홍길동',
  '관계': ['KNOWS(2012)', 'KNOWS(2014)', 'KNOWS(2020)'],
  '도착점': '서윤주'},
 {'시작점': '홍길동',
  '관계': ['KNOWS(2012)', 'KNOWS(2014)', 'KNOWS(2017)'],
  '도착점': '권두현'},
 {'시작점': '홍길동', '관계': ['KNOWS(2015)'], '도착점': '이영희'},
 {'시작점': '홍길동', '관계': ['KNOWS(2015)', 'KNOWS(2020)'], '도착점': '서윤주'},
 {'시작점': '홍길동', '관계': ['KNOWS(2015)', 'KNOWS(2017)'], '도착점': '권두현'},
 {'시작점': '홍길동',
  '관계': ['KNOWS(2015)', 'KNOWS(2017)', 'KNOWS(2024)'],
  '도착점': '서윤주'},
 {'시작점': '홍길동', '관계': ['KNOWS(2010)'], '도착점': '김철수'},
 {'시작점': '홍길동', '관계': ['KNOWS(2010)', 'KNOWS(2018)'], '도착점': '박규리'},
 {'시작점': '홍길동',
  '관계': ['KNOWS(2010)', 'KNOWS(2018)', 'KNOWS(2024)'],
  '도착점': '김하경'}]

In [ ]:
# *2 표기법은 정확히 2단계 친구 관계 조회합니다.
# 예 : 홍길동 -> 이영희 -> 서윤주 (홍길동의 친구의 친구)
cypher_query = """
MATCH (p1: Person {name: '홍길동'})-[:KNOWS*2]->(p2:Person)

RETURN p1.name AS 시작점, p2.name AS 도착점
"""

graph.query(cypher_query)

[{'시작점': '홍길동', '도착점': '이영희'},
 {'시작점': '홍길동', '도착점': '권두현'},
 {'시작점': '홍길동', '도착점': '서윤주'},
 {'시작점': '홍길동', '도착점': '박규리'},
 {'시작점': '홍길동', '도착점': '김하경'}]

In [13]:
# [:KNOWS*] 표기법은 KNOWS 관계를 통해 연결된 모든 깊이의 경로를 탐색합니다.
# 단계 제한이 없으므로 1단계,2단계,3단계 이상 모두 포함됩니다.
# 방향성이 있는 -> 연산자를 사용하여 홍길동에서 출발하는 단방향 관계만 조회합니다.

cypher_query = """
MATCH (p1: Person {name: '홍길동'})-[r:KNOWS*]->(p2:Person)

RETURN p1.name AS 시작점, p2.name AS 도착점, size(r) AS 관계의_수
"""

graph.query(cypher_query)

[{'시작점': '홍길동', '도착점': '변지민', '관계의_수': 1},
 {'시작점': '홍길동', '도착점': '김하경', '관계의_수': 2},
 {'시작점': '홍길동', '도착점': '이영희', '관계의_수': 2},
 {'시작점': '홍길동', '도착점': '서윤주', '관계의_수': 3},
 {'시작점': '홍길동', '도착점': '권두현', '관계의_수': 3},
 {'시작점': '홍길동', '도착점': '서윤주', '관계의_수': 4},
 {'시작점': '홍길동', '도착점': '이영희', '관계의_수': 1},
 {'시작점': '홍길동', '도착점': '서윤주', '관계의_수': 2},
 {'시작점': '홍길동', '도착점': '권두현', '관계의_수': 2},
 {'시작점': '홍길동', '도착점': '서윤주', '관계의_수': 3},
 {'시작점': '홍길동', '도착점': '김철수', '관계의_수': 1},
 {'시작점': '홍길동', '도착점': '박규리', '관계의_수': 2},
 {'시작점': '홍길동', '도착점': '김하경', '관계의_수': 3}]

### 3. 최단경로 (Shortest path)

In [ ]:
#홍길동과 박규리 사이의 최단 경로 조회

cypher_query="""
MATCH p=shortestPath((p1:Person {name:'홍길동'})-[*]-(p2:Person {name:'박규리'}))

//[*] 표기법은 모든 유형의 관계를 고려하며, 방향성 없이 양방향으로 탐색한다.
//이는 KNOWS, LIKES 등 모든 관계 타입을 포함한다.

RETURN [node IN nodes(p) | node.name] AS 경로
"""
graph.query(cypher_query)

[{'경로': ['홍길동', '김철수', '박규리']}]

In [ ]:
#KNOWS 관계만을 사용하여 변지민과 박규리 사이의 최단 경로 조회

cypher_query="""
MATCH p=shortestPath((p1:Person {name:'변지민'})-[:KNOWS*]-(p2:Person {name:'박규리'}))

//[*] 표기법은 모든 유형의 관계를 고려하며, 방향성 없이 양방향으로 탐색한다.
//이는 KNOWS, LIKES 등 모든 관계 타입을 포함한다.

RETURN [node IN nodes(p) | node.name] AS 경로
"""
graph.query(cypher_query)

[{'경로': ['변지민', '김하경', '박규리']}]

In [ ]:
#경로 길이 제한이 있는 최단 경로
#최대 3단계 관계 내에서 홍길동과 김하경 사이의 최단 경로 조회
#이는 너무 긴 경로를 방지하고 가까운 사회적 연결만 찾고자 할때 유용합니다.

cypher_query="""
MATCH p=shortestPath((p1:Person {name:'홍길동'})-[*..3]-(p2:Person {name:'김하경'}))

//[*] 표기법은 모든 유형의 관계를 고려하며, 방향성 없이 양방향으로 탐색한다.
//이는 KNOWS, LIKES 등 모든 관계 타입을 포함한다.

RETURN [node IN nodes(p) | node.name] AS 경로
"""
graph.query(cypher_query)

[{'경로': ['홍길동', '변지민', '김하경']}]

### 4. 집계함수 (Aggregation Functions)

In [34]:
# 사람 수 계산
cypher_query="""
//전체 Person 노드 수 계산
MATCH (p:Person)

RETURN COUNT(p) AS 전체사람수
"""

graph.query(cypher_query)

[{'전체사람수': 8}]

In [36]:
# 도시 별 사람 수 계산
cypher_query="""
//도시별 Person 노드 수 계산
MATCH (p:Person)

RETURN p.city AS 도시, COUNT(p) AS 사람수
ORDER BY 사람수 DESC
"""

graph.query(cypher_query)

[{'도시': '서울', '사람수': 3},
 {'도시': '수원', '사람수': 2},
 {'도시': '구미', '사람수': 2},
 {'도시': '부산', '사람수': 1}]

In [37]:
# 평균 나이 계산
cypher_query="""
MATCH (p:Person)
RETURN AVG(p.age) AS 평균나이
"""
graph.query(cypher_query)

[{'평균나이': 29.5}]

In [38]:
# 관심분야별 좋아하는 사람 수
cypher_query="""
MATCH (p:Person)-[:LIKES]->(i:Interest)

RETURN i.name AS 관심분야, COUNT(p) AS 사람수
ORDER BY 사람수 DESC
"""

graph.query(cypher_query)

[{'관심분야': '영화', '사람수': 2},
 {'관심분야': '강아지', '사람수': 2},
 {'관심분야': '고양이', '사람수': 2},
 {'관심분야': '뮤지컬', '사람수': 2},
 {'관심분야': '여행', '사람수': 2}]

In [41]:
# 가장 높은 평점의 장소 찾기
cypher_query="""
// Person 노드에서 VISITED 관계를 통해 Location 노드로 연결된 패턴을 찾는다.
MATCH (p:Person)-[v:VISITED]->(l:Location)

RETURN l.name AS 장소, AVG(v.rating) AS 평균평점
ORDER BY 평균평점 DESC

//평점이 가장 높은 장소 한개만 출력한다.
LIMIT 1
"""

graph.query(cypher_query)

[{'장소': '광화문', '평균평점': 4.5}]

### 5. WITH절
* 중간 결과를 다음 쿼리 부분에 전달할 수 있게 해줍니다.

In [43]:
# 기본 필터링
cypher_query="""
MATCH (p:Person)
WHERE p.age >=20

//WITH p 는 이전 단계에서 필터링 된 결과(p)를 다음 쿼리 부분으로 전달한다.
//WITH 절은 쿼리의 중간 결과를 다음 단게로 파이프라인처럼 전달하는 역할을 한다.
WITH p

WHERE p.name CONTAINS '김'
RETURN p.name AS 이름, p.age AS 나이
"""

graph.query(cypher_query)

[{'이름': '김철수', '나이': 28}, {'이름': '김하경', '나이': 29}]

In [ ]:
# 집계 결과를 중간 처리한 후 다시 활용
cypher_query="""
MATCH (p:Person)
WITH p.city AS 도시, AVG(p.age) AS 평균나이
WHERE 평균나이 >=30
RETURN 도시, 평균나이
ORDER BY 평균나이 DESC
"""

graph.query(cypher_query)

[{'도시': '부산', '평균나이': 35.0}, {'도시': '구미', '평균나이': 31.0}]

In [48]:
# 복잡한 집계 작업
cypher_query="""
MATCH (p:Person)

//OPTIONAL MATCH를 사용하여 방문한 장소가 없는 사람도 결과에 포함시킨다.
//방문 장소가 없는 사람의 경우 l은 NULL이 된다.
OPTIONAL MATCH (p)-[:VISITED]->(l:Location)

//각 사람(p) 별로 방문한 고유 장소(l)의 수를 계산한다.
//이 중간 결과를 '방문장소수'라는 변수에 저장하고 다음 쿼리부분으로 전달한다.
WITH p, Count(DISTINCT l) AS 방문장소수

OPTIONAL MATCH (p)-[:LIKES]->(i:Interest)
WITH p, 방문장소수, COUNT(DISTINCT i) AS 관심분야수

WHERE 방문장소수 >=2 AND 관심분야수 >=2
RETURN p.name AS 이름, 방문장소수, 관심분야수
"""

graph.query(cypher_query)

[{'이름': '홍길동', '방문장소수': 2, '관심분야수': 2}]

In [ ]:
# 복잡한 패턴 결합
# 같은 관심사를 가진 친구의 친구 찾기
cypher_query="""
MATCH (p1: Person)-[:LIKES]->(i:Interest {name:'영화'})

//앞서 찾은 영화에 관심사가 있는 사람 p1과 2단계로 연결되어있는 다른사람 p2를 찾는다.
// 그 사람(p2)이 같은 영화 관심사 (i)ㄹㄹ 가지고 있는지 확인한다.
// p2에서 LIKES 관계를 통해서 앞서 찾은 동일한 interest 노드(i)로 연결되어 있어야 한다.
MATCH (p1)-[:KNOWS*2]->(p2:Person)-[:LIKES]->(i)

//p1과 p2가 서로다른 사람인지 확인 -> 자기 자신으로 돌아오는 경로는 제외시키기 위함
WHERE p1<>p2
RETURN p1.name AS 시작사람, p2.name AS 연결된사람
"""
graph.query(cypher_query)

[{'시작사람': '홍길동', '연결된사람': '권두현'}]

In [ ]:
# 친구 관계와 공통 관심사 결합
cypher_query="""
//홍길동과 같은 관심사를 가진 모든 사람들을 식별한다.
MATCH (p1:Person {name:'홍길동'})-[:LIKES]->(i:Interest)<-[:LIKES]-(p2:Person)

// 홍길동과 직접적으로 알고있는 사이가 아닌 경우만 찾아냄
WHERE NOT (p1)-[:KNOWS]-(p2) 
RETURN DISTINCT p2.name AS 이름, i.name AS 공통관심사
"""
graph.query(cypher_query)

[{'이름': '권두현', '공통관심사': '영화'}, {'이름': '김하경', '공통관심사': '여행'}]

In [ ]:
# 최단 경로와 집계 함수 결합
cypher_query="""
// 홍길동과 다른 사람들까지의 최단 경로를 찾는다.
MATCH p=shortestPath((start:Person {name:'홍길동'})-[:KNOWS*]-(other:Person))
WHERE start <> other
WITH other, length(p) AS 거리
RETURN 거리, COUNT(other) AS 사람수

// 결과를 거리 값에 따라 오름차순으로 정렬한다.
ORDER BY 거리
"""

graph.query(cypher_query)

[{'거리': 1, '사람수': 3}, {'거리': 2, '사람수': 4}]